In [ ]:
%load_ext autoreload
%autoreload 

In [1]:
import os
import shutil
import re
import pickle
from src.data_loader import extract_latex_dependencies, build_resume_context, flatten_pydantic, rebase_dependency_paths, convert_profile_latex
from src.llm_tools import enrich_file_metadata, extract_jd_features, tailor_profile_and_highlights


/Users/Bardia/Coding/LangChain/lib/python3.14/site-packages/langchain_core/_api/deprecation.py:26: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1


In [2]:
# check for the cache file containing all the dependencies and load it if it exists
cache_dir = "./cache"
# check the validity of the cache directory
if not os.path.exists(cache_dir):
    raise FileNotFoundError(f"The directory was not found: ' {cache_dir} ' ")

cache_file_name = "resume_metadata.pkl"
cache_path = os.path.join(cache_dir, cache_file_name)

if os.path.exists(cache_path): 
    with open(cache_path, 'rb') as f: 
        dep_list = pickle.load(f)
        print("File loaded")
#else proccess the latex main file for the dependencies list
else: 
    rsm_main_path = "./Rsm/main/Resume_Bardia_Azami/resume-general/Bardia-Azami-Resume.tex"
    #check the validity of file path 
    if not os.path.exists(rsm_main_path): 
        raise FileNotFoundError(f"The directory was not found: ' {rsm_main_path} ' ")
    #unproccesed dependencies list
    dep_unprc = extract_latex_dependencies(rsm_main_path)
    #enrich the dependencies list 
    dep_list = enrich_file_metadata(dep_unprc)
    print("dependencies extracted")
    #save the list 
    with open(cache_path, 'wb') as f: 
        pickle.dump(dep_list, f)
        print("The proccesed dependencies saved")


File loaded


In [3]:
# Define the directories
jd_dir = "./Rsm/jd"
rsm_dir = "./Rsm/main/Resume_Bardia_Azami"
rsm_tailored_path ="./Rsm/tailored"
for path in [jd_dir, rsm_dir, rsm_tailored_path]:
    if not os.path.exists(path):
        raise FileExistsError(f"could not find the job description file:{jd_dir} ")

# list all .tex files in jd directory
tex_files = [f for f in os.listdir(jd_dir) if f.endswith(".txt")]

# create destination folders for tailored resumes and copy the main resume for making adjusments
for tex_file in tex_files: 
    target_dir = os.path.join( rsm_tailored_path , os.path.splitext(tex_file)[0] )
    os.makedirs(target_dir, exist_ok=True)

    destination = os.path.join(target_dir, os.path.basename(rsm_dir))
    if not os.path.exists(destination):
        shutil.copytree(rsm_dir, destination)

    #update dep list for the tailored resume path 
    new_dep_list = rebase_dependency_paths(dep_list , destination, rsm_dir)



Rebasing paths...
FROM: ./Rsm/main/Resume_Bardia_Azami
TO:   ./Rsm/tailored/GPTZero(Machine Learning Intern)/Resume_Bardia_Azami



In [4]:
new_dep_list

[{'section': 'CONTACT_INFORMATION',
  'full_path': './Rsm/tailored/GPTZero(Machine Learning Intern)/Resume_Bardia_Azami/components/background.tex',
  'title': 'Contact Information'},
 {'section': 'PROFILE',
  'full_path': './Rsm/tailored/GPTZero(Machine Learning Intern)/Resume_Bardia_Azami/components/Profile.tex',
  'title': 'Professional Summary'},
 {'section': 'HIGHLIGHT OF QUALIFICATIONS',
  'full_path': './Rsm/tailored/GPTZero(Machine Learning Intern)/Resume_Bardia_Azami/components/Qualifications/QualificationsHighlight.tex',
  'title': 'Qualifications Summary'},
 {'section': 'TECHNICAL EXPERIENCE',
  'full_path': './Rsm/tailored/GPTZero(Machine Learning Intern)/Resume_Bardia_Azami/components/experiences/BSS.tex',
  'title': 'Computer Vision Engineer - Behyar Sanaat Sepahan'},
 {'section': 'TECHNICAL EXPERIENCE',
  'full_path': './Rsm/tailored/GPTZero(Machine Learning Intern)/Resume_Bardia_Azami/components/experiences/Researcher-UI.tex',
  'title': 'AI Researcher - Univ of Isfahan'

In [ ]:
#TEMP
jd_path = "./Rsm/jd/GPTZero(Machine Learning Intern).txt"

if not os.path.exists(jd_path):
    raise FileNotFoundError(f"File not found: {jd_file_path}")
        
try:
    with open(jd_path, 'r', encoding='utf-8') as f:
        jd = f.read()
except Exception as e:
    raise IOError(f"Error reading file: {e}")
    
jd_kw = extract_jd_features(jd)

#TEMP

In [ ]:
jd_kw_ls = flatten_keywords(jd_kw , ['domain_knowledge', 'technical_stack', 'tools_and_platforms', 'soft_skills'])
print(len(jd_kw_ls))

In [ ]:
rsm = build_resume_context(dep_list) 

In [ ]:
data = tailor_profile_and_highlights(rsm, jd, jd_kw_ls)

In [ ]:
profile, hl_qualifications = convert_profile_latex(data)